# Evaluating Observed Flood Extent from the GFM against Forecast Flood Extent from the EFAS Rapid Flood Mapping layer

With this notebook, we show how to use the new GFM STAC service to
derive the maximum flood extent for a flood event, extract the corresponding
forecast product from the EFAS Rapid Flood Mapping layer and perform
a simple evaluation.

This example notebook is structured as follows:

0. (Import and install the necessary python packages)
1. Download data from the GFM STAC service (ensemble flood extent, reference water, exclusion mask)
2. Download the EFAS Rapid Flood Mapping data
3. Evaluate EFAS layer against the GFM data - exclude areas either within reference water mask or exclusion masks

We present an example analysis of the flooding that occurred in south-west 
Poland between the 18th - 28th September 2024

Please note you must have an EFAS account to access the data used in
this notebook

## 0. Import the necessary python modules

In [1]:
# Some necessary imports

from pystac_client import Client
from datetime import datetime
from odc import stac as odc_stac
import pyproj
import rioxarray # noqa
import xarray as xr
from shapely.geometry import box
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds

import numpy as np
import requests
from zipfile import ZipFile

## 1. Extract Observed Flood Extent Data using the GFM STAC service

Firstly define the GFM product you wish to download.

Then define a time range of interest and the coordinates of the area of
interest (aoi).

These parameters are passed to the GFM STAC service to find the available data.

In [ ]:
# Define bounding box (West, South, East, North)
aoi = box(16.77, 49.91, 18.62, 51.25)

# Define time range
time_range = (datetime(2024, 9, 18), datetime(2024, 9, 28))

# EODC STAC API URL
api_url = "https://stac.eodc.eu/api/v1"
eodc_catalog = Client.open(api_url)

# Define search query using pystac_client
search = eodc_catalog.search(
    max_items=1000,
    collections="GFM",
    intersects=aoi,
    datetime=time_range
)

# Get all found items
items = search.item_collection()
print("We found", len(items), "items, that match our filter criteria.")

# Retrieve the CRS from one of the found items
crs_equi7 = pyproj.CRS.from_wkt(items[0].properties["proj:wkt2"])

### Now the returned data are loaded into an xarray dataset in the Equi7Grid projection.
Additionally, the sum over the time dimension is carried out.

In [ ]:
# Define the asset names you want to use 
asset_names = ["ensemble_flood_extent", "reference_water_mask", "exclusion_mask"]

# By setting chunks, the odc library "lazy loads" the data. -1 for time means
# that all time steps are included in one chunk. Reduce the chunk size for x
# and/or y if your kernel crashes due to out of memory issues
xx = odc_stac.load(
    items, 
    bbox=aoi.bounds,
    crs=crs_equi7,
    bands=asset_names,
    dtype="uint8",
    chunks={"x": 2000, "y": 2000, "time": -1}, 
    resolution=20)

# Only include data which is not nodata and not 0
nodata = 255
xx = xx.where((xx != nodata) & (xx != 0))

# Create the sum over the time dimension
# This is done for all specified assets
data = xx.sum(dim="time").astype("uint8")

Once the above cell is loaded, we advise to save the data on disk for later use.
With that, you do not need to download the same data everytime you start this notebook.

In [3]:
# Optional: Save the "raw" data as ZARR data store for later use
# xx.to_zarr("./gfm_data.zarr")

# Read data from ZARR data store once saved
# xx = xr.open_zarr("./gfm_data.zarr")

# ZARR is currently not writing CRSs into data stores. Add it again after
# opening the data store
# xx.rio.write_crs(crs_equi7, inplace=True)

# nodata = 255

# Create the sum over the time dimension
# This is done for all specified assets
# data = xx.sum(dim="time").astype("uint8")

### Set valid values to 1 for each individual layer and save it as GTiff

In [4]:
# Extract maximum flood extent
data["ensemble_flood_extent"] = data["ensemble_flood_extent"].where((data["ensemble_flood_extent"] < 1) | (data["ensemble_flood_extent"] > nodata), 1)

# Reference water mask
data["reference_water_mask"] = data["reference_water_mask"].where((data["reference_water_mask"] < 1) | (data["reference_water_mask"]  > nodata), 1)

# # Exclusion mask
data["exclusion_mask"] = data["exclusion_mask"].where((data["exclusion_mask"] < 1) | (data["exclusion_mask"] > nodata), 1)

# Save as Raster in Equi7Grid
data["ensemble_flood_extent"].rio.to_raster("ensemble_flood_extent_equi7.tif", compress="LZW")
data["reference_water_mask"].rio.to_raster("reference_water_mask_equi7.tif", compress="LZW")
data["exclusion_mask"].rio.to_raster("exclusion_mask_equi7.tif", compress="LZW")

In [5]:
# Get the bounding coordinates of the GFM data
dx = data.x.values[1]-data.x.values[0]
dy = data.y.values[1]-data.y.values[0]

xmin = data.x.values.min() - (dx/2)
xmax = data.x.values.max() + (dy/2)

ymin = data.y.values.min() - abs(dy/2)
ymax = data.y.values.max() + abs(dy/2)

## 2. Download Forecasted Flood Extent from the EFAS Rapid Flood Mapping layer

Specify the date of the forecast for which you wish to retrieve data. You
will also need to retrieve your efas web token, which you can generate by
logging in to this website https://european-flood.emergency.copernicus.eu/en/efas-web-services
and copying the web token shown in the table.

In [6]:
fcast_date = '2024091800'  # date of the forecast as string in YYYYMMDDHH format
efas_token = 'your_efas_token'  # generate token at https://european-flood.emergency.copernicus.eu/en/efas-web-services
#                                                the token needs to be regenerated after every use!

url_rfm = f'https://european-flood.emergency.copernicus.eu/api/fms/download/efas/RapidFloodMapping/{fcast_date[0:4]}-{fcast_date[4:6]}-{fcast_date[6:8]}T{fcast_date[8:10]}:00Z?token={efas_token}'

#### Download the forecast flood extent file from the EFAS website, load it in then clip to AOI

This will save a zip file into the same folder where you have saved
this notebook. An ESRI shapefile containing the forecast flood 
extent will be extracted from this zip file into the same folder.

In [ ]:
response = requests.get(url_rfm)
out_zip = f'RFM_{fcast_date}.zip'
with open(out_zip, 'wb') as file:
  file.write(response.content)

with ZipFile(out_zip, 'r') as zip_ref:
  zip_ref.extractall('.')

#### Load the Forecasted Flood Extent file

The forecast flood extent shapefile is loaded as a GeoPandas
GeoDataFrame. It is then clipped to the extent of the aoi 
which you defined previously

In [7]:
rfm_file = f'FloodMask{fcast_date}.shp'
rfm_data = gpd.read_file(rfm_file)

# Clip the Rapid Flood Mapping to your drawn area
rfm_intersect = gpd.clip(rfm_data, gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[aoi]))

## 3. Compare GFM and RFM Data

In this part the observed flood extent from the GFM and forecast
flood extent from EFAS are compared to identify areas of agreement
and disagreement.

#### Firstly convert the Forecast Flood Extent file from an ESRI Shapefile into a grid

We use rasterio.rasterize() to convert the shapefile onto the same grid as the
GFM maximum flood extent which we re-projected onto the Equi7Grid projection.

In [8]:
gfm_flood = data["ensemble_flood_extent"].values

transform = from_bounds(xmin, ymin, xmax, ymax, gfm_flood.shape[1], gfm_flood.shape[0])

rfm_intersect_equi7 = rfm_intersect.to_crs(crs_equi7)

# Get geometry and corresponding value from the GeoDataFrame
shapes = ((geom, 1) for geom in rfm_intersect_equi7.geometry)

# Rasterize the geometries into the numpy array
rfm_raster = rasterize(
    shapes,
    out_shape=gfm_flood.shape,
    transform=transform,
    fill=0, 
    dtype=np.uint8
)

#### Compare the Observed and Forecast Flood Extent Grids

Use a contingency table approach to classify each grid cell as one of the following:
1. Hit - where flooding is present in both the GFM & EFAS forecast
2. False alarm - flooding only present in the EFAS forecast
3. Miss - flooding only present in the GFM observations
4. Correct negative - no flooding present in either GFM or EFAS data

We exclude areas which are either a) in the reference water mask, or b) in the exclusion mask

From this classification, we compute the following performance scores:
1. CSI (Critical Success Index), range 0-1 where 1 = perfect agreement between forecast and observations
2. False Alarm Ration, range 0-1 where 0 is optimal, shows the fraction of grid cells which were forecasted as being flooded which were incorrect
3. Hit Rate, range 0-1 where 1 is optimal, shows the fraction of observed flooded grid cells which were correctly forecasted

In [ ]:
# compare GFM and RFM data

gfm_rwm = data["reference_water_mask"].values
gfm_em = data["exclusion_mask"].values

# compute the total number of hits, false alarms, misses and correct negatives
tp = np.where((gfm_flood == 1) & (rfm_raster == 1) & (gfm_rwm < 1) & (gfm_em == 0))[0].shape[0] # hits
fp = np.where((gfm_flood == 0) & (rfm_raster == 1) & (gfm_rwm < 1) & (gfm_em == 0))[0].shape[0] # false alarms
fn = np.where((gfm_flood == 1) & (rfm_raster == 0) & (gfm_rwm < 1) & (gfm_em == 0))[0].shape[0] # misses
tn = np.where((gfm_flood == 0) & (rfm_raster == 0) & (gfm_rwm < 1) & (gfm_em == 0))[0].shape[0] # correct negatives

csi = (tp / (tp + fp + fn))  # critical success index (CSI)
far = (fp / (tp + fp))  # false alarm ration (FAR)
hr = (tp / (tp + fn))  # hit rate (HR)

print(f'CSI: {"%.2f" % csi}')
print(f'False Alarm Ratio: {"%.2f" % far}')
print(f'Hit Rate: {"%.2f" % hr}')

# Create a grid for the evaluation results to show on a map
# create a new grid which shows whether each grid cell is a hit, false alarm or miss
eval_grid = np.zeros(gfm_flood.shape, dtype=np.uint8)
eval_grid[(gfm_flood == 1) & (rfm_raster == 1) & (gfm_rwm < 1) & (gfm_em ==0)] = 1  # Hits
eval_grid[(gfm_flood == 1) & (rfm_raster == 0) & (gfm_rwm < 1) & (gfm_em ==0)] = 2  # Misses
eval_grid[(gfm_flood == 0) & (rfm_raster == 1) & (gfm_rwm < 1) & (gfm_em ==0)] = 3  # False alarms

In [10]:
# Save RFM data as raster
with rasterio.open('./rfm_data_equi7.tif', 'w', driver='GTiff', height=rfm_raster.shape[0],
                   width=rfm_raster.shape[1], count=1, dtype=rfm_raster.dtype,
                   crs=crs_equi7, transform=transform, compress="LZW") as dst:
    dst.write(rfm_raster, 1)

# Save RFM GFM evaluation results as raster
with rasterio.open('./rfm_gfm_eval_equi7.tif', 'w', driver='GTiff', height=eval_grid.shape[0],
                   width=eval_grid.shape[1], count=1, dtype=eval_grid.dtype,
                   crs=crs_equi7, transform=transform, compress="LZW") as dst:
    dst.write(eval_grid, 1)


## Conclusions

This notebook shows how you can download specific GFM data 
using the GFM STAC service and the EFAS Rapid Flood Mapping forecast 
layer and compare the data, with the idea of evaluating the 
performance of the forecast layer.

In general it could be seen that:
* The forecasted inundation extent generally over-predicted when compared to GFM
* Because forecast data is coarser resolution
    * Smoothed representation of floodplain topography, e.g. doesn't represent levees
* Therefore you need to be careful regarding the precision for which the forecast information can be used
    * It can highlight general floodplain areas at risk but be careful when identifying specific towns/cities affected